In [1]:
import os
import sys

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "mysite.settings")

from django.core.management import execute_from_command_line

execute_from_command_line(sys.argv)


Unknown command: '-f'
Type '__main__.py help' for usage.


SystemExit: 1

/Users/youk/workspace/djangogirls-tutorial/myvenv/lib/python3.4/site-packages/IPython/core/interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)



In [2]:
from blog.models import Post

In [21]:
Post.objects.all()

[<Post: Sample title>, <Post: Hello world!>, <Post: Sample title>, <Post: How Google works.>]

In [8]:
from django.contrib.auth.models import User
me = User.objects.get(username='admin')

In [9]:
Post.objects.create(author=me, title='Sample title', text='Test')

<Post: Sample title>

In [10]:
Post.objects.filter(author=me)

[<Post: Sample title>, <Post: Hello world!>, <Post: Sample title>]

In [11]:
Post.objects.filter( title__contains='title' )

[<Post: Sample title>, <Post: Sample title>]

In [12]:
from django.utils import timezone
Post.objects.filter(published_date__lte=timezone.now())

[]

In [14]:
post = Post.objects.get(title="Hello world!")

In [15]:
post.publish()

In [16]:
Post.objects.filter(published_date__lte=timezone.now())

[<Post: Hello world!>]

In [17]:
Post.objects.order_by('created_date')

[<Post: Sample title>, <Post: Hello world!>, <Post: Sample title>]

In [18]:
Post.objects.order_by('-created_date')

[<Post: Sample title>, <Post: Hello world!>, <Post: Sample title>]

In [19]:
Post.objects.filter(published_date__lte=timezone.now()).order_by('published_date')

[<Post: Hello world!>]

In [20]:
post2 = Post(author=me, title="How Google works.", text="Descript about how google works.")
post2.publish()

= = = = = = = = = = = = = = = = = = = = = = = = = = =
# ORM Exercise

= = = = = = = = = = = = = = = = = = = = = = = = = = =


In [16]:
from blog.models import Blog
b = Blog(name='Beatles Blog', tagline='All the latest Beatles news.')
b.save()

In [18]:
b2 = Blog(name='Cheddar Talk')
b2.save()

In [19]:
Blog.objects.all()

[<Blog: Beatles Blog>, <Blog: b5>, <Blog: b5>, <Blog: Beatles Blog>, <Blog: Cheddar Talk>]

In [9]:
b5 = Blog(name='b5', tagline='b5 instance')
b5.save()

In [33]:
from django.utils import timezone
e = Entry(blog=b5, headline="entry", pub_date=timezone.now(), mod_date=timezone.now(), n_comments=3, n_pingbacks=4, rating=4)
e.save()

In [20]:
from blog.models import Entry
entry = Entry.objects.get(pk=1) #pk means primary key
cheese_blog = Blog.objects.get(name="Cheddar Talk")
entry.blog = cheese_blog
entry.save()

In [22]:
from blog.models import Author
john = Author.objects.create(name="John")
paul = Author.objects.create(name="Paul")
george = Author.objects.create(name="George")
ringo = Author.objects.create(name="Ringo")
entry.authors.add(john, paul, george, ringo)

# Retrieving objects

To retrieve objects from your database, construct a QuerySet via a Manager on your model class.

A QuerySet represents a collection of objects from your database. It can have zero, one or many filters. Filters narrow down the query results based on the given parameters. In SQL terms, a QuerySet equates to a SELECT statement, and a filter is a limiting clause such as WHERE or LIMIT.

You get a QuerySet by using your model’s Manager. Each model has at least one Manager, and it’s called objects by default.

In [31]:
Entry.objects.filter(pub_date__year=2006)

[]

In [34]:
Entry.objects.all()

[<Entry: >, <Entry: entry>]

In [28]:
# Chaining filter

import datetime
Entry.objects.filter(
    headline__startswith='What'
).exclude(
    pub_date__gte=datetime.date.today()
).filter(
    pub_date__gte=datetime.datetime(2005, 1, 30)
)

[]

### QuerySets are lazy¶

QuerySets are lazy – the act of creating a QuerySet doesn’t involve any database activity.  
You can stack filters together all day long, and Django won’t actually run the query until the QuerySet is evaluated.  


In [30]:
Blog.objects.get(name__contains='dar')

<Blog: Cheddar Talk>

### Lookups that span relationships

Django offers a powerful and intuitive way to “follow” relationships in lookups, taking care of the SQL JOINs for you automatically, behind the scenes. To span a relationship, just use the field name of related fields across models, separated by double underscores, until you get to the field you want.

In [37]:
Entry.objects.filter(blog__name='b5')

[<Entry: entry>]

### Filters can reference fields on the model

In the examples given so far, we have constructed filters that compare the value of a model field with a constant. 
**But what if you want to compare the value of a model field with another field on the same model?**
**만약에 모델 필드의 값을 같은 모델의 다른 필드와 비교하려면 어떻게 해야 할까?**

Django provides F expressions to allow such comparisons. Instances of F() act as a reference to a model field within a query. These references can then be used in query filters to compare the values of two different fields on the same model instance.

For example, to find a list of all blog entries that have had more comments than pingbacks, we construct an F() object to reference the pingback count, and use that F() object in the query:

~~~
from django.db.models import F
Entry.objects.filter(n_comments__gt=F('n_pingbacks'))
~~~

Django supports the use of addition, subtraction, multiplication, division, modulo, and power arithmetic with F() objects, both with constants and with other F() objects. To find all the blog entries with more than twice as many comments as pingbacks, we modify the query:



In [40]:
from django.db.models import F
Entry.objects.filter(n_pingbacks=F('rating'))

[<Entry: >, <Entry: entry>]

In [42]:
Entry.objects.filter(n_comments__lt = F('n_pingbacks') * 2)

[<Entry: >, <Entry: entry>]

In [45]:
Entry.objects.filter(authors__name=F('blog__name'))

[]

In [46]:
Blog.objects.all()

[<Blog: Beatles Blog>, <Blog: b5>, <Blog: b5>, <Blog: Beatles Blog>, <Blog: Cheddar Talk>]

In [44]:
Blog.objects.filter(pk__lt=3)

[<Blog: Beatles Blog>, <Blog: b5>]

### Complex lookups with Q objects

Keyword argument queries – in filter(), etc. – are “AND”ed together. If you need to execute more complex queries (for example, queries with OR statements), you can use Q objects.

A Q object (django.db.models.Q) is an object used to encapsulate a collection of keyword arguments. These keyword arguments are specified as in “Field lookups” above.

For example, this Q object encapsulates a single LIKE query:

In [48]:
from django.db.models import Q
Q(question__startswith='What')

In [51]:
blog = Blog(name='My blog', tagline='Blogging is easy')
blog.save() # blog.pk == 1

blog.pk = None
blog.save() # blog.pk == 2
print(blog)

My blog


### Updating multiple objects at once

Sometimes you want to set a field to a particular value for all the objects in a QuerySet. 
You can do this with the update() method. 

Be aware that the update() method is converted directly to an SQL statement. It is a bulk operation for direct updates. It doesn’t run any save() methods on your models, or emit the pre_save or post_save signals (which are a consequence of calling save()), or honor the auto_now field option. If you want to save every item in a QuerySet and make sure that the save() method is called on each instance, you don’t need any special function to handle that. Just loop over them and call save( ):

~~~
for item in my_queryset:
    item.save()
~~~